In [33]:
from IPython.core.display import HTML

with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Chess core functions

### Imports

In [34]:
import chess
import chess.polyglot
import math
import random
import heapq

## Auxiliary functions

### Hash function

In [35]:
def zobrist_hash(board):
    return chess.polyglot.zobrist_hash(board)

### Piece-Square Tables
Values are taken from: https://www.chessprogramming.org/Simplified_Evaluation_Function#Piece-Square_Tables  
These tables represent the values for the black pieces.  
For the king, two different tables are used, depending on wether the board is in the endgame or not (for further explanation, see below).

In [36]:
PAWN_VALUES = [0, 0, 0, 0, 0, 0, 0, 0,
               50, 50, 50, 50, 50, 50, 50, 50,
               10, 10, 20, 30, 30, 20, 10, 10,
               5, 5, 10, 25, 25, 10, 5, 5,
               0, 0, 0, 20, 20, 0, 0, 0,
               5, -5, -10, 0, 0, -10, -5, 5,
               5, 10, 10, -20, -20, 10, 10, 5,
               0, 0, 0, 0, 0, 0, 0, 0]

KNIGHT_VALUES = [-50, -40, -30, -30, -30, -30, -40, -50,
                 -40, -20, 0, 0, 0, 0, -20, -40,
                 -30, 0, 10, 15, 15, 10, 0, -30,
                 -30, 5, 15, 20, 20, 15, 5, -30,
                 -30, 0, 15, 20, 20, 15, 0, -30,
                 -30, 5, 10, 15, 15, 10, 5, -30,
                 -40, -20, 0, 5, 5, 0, -20, -40,
                 -50, -40, -30, -30, -30, -30, -40, -50]

BISHOP_VALUES = [-20, -10, -10, -10, -10, -10, -10, -20,
                 -10, 0, 0, 0, 0, 0, 0, -10,
                 -10, 0, 5, 10, 10, 5, 0, -10,
                 -10, 5, 5, 10, 10, 5, 5, -10,
                 -10, 0, 10, 10, 10, 10, 0, -10,
                 -10, 10, 10, 10, 10, 10, 10, -10,
                 -10, 5, 0, 0, 0, 0, 5, -10,
                 -20, -10, -10, -10, -10, -10, -10, -20]

ROOK_VALUES = [0, 0, 0, 0, 0, 0, 0, 0,
               5, 10, 10, 10, 10, 10, 10, 5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               -5, 0, 0, 0, 0, 0, 0, -5,
               0, 0, 0, 5, 5, 0, 0, 0]

QUEEN_VALUES = [-20, -10, -10, -5, -5, -10, -10, -20,
                -10, 0, 0, 0, 0, 0, 0, -10,
                -10, 0, 5, 5, 5, 5, 0, -10,
                -5, 0, 5, 5, 5, 5, 0, -5,
                0, 0, 5, 5, 5, 5, 0, -5,
                -10, 5, 5, 5, 5, 5, 0, -10,
                -10, 0, 5, 0, 0, 0, 0, -10,
                -20, -10, -10, -5, -5, -10, -10, -20]

KING_VALUES_MIDDLEGAME = [-30, -40, -40, -50, -50, -40, -40, -30,
                          -30, -40, -40, -50, -50, -40, -40, -30,
                          -30, -40, -40, -50, -50, -40, -40, -30,
                          -30, -40, -40, -50, -50, -40, -40, -30,
                          -20, -30, -30, -40, -40, -30, -30, -20,
                          -10, -20, -20, -20, -20, -20, -20, -10,
                          20, 20, 0, 0, 0, 0, 20, 20,
                          20, 30, 10, 0, 0, 10, 30, 20]

KING_VALUES_ENDGAMAE = [-50, -40, -30, -20, -20, -30, -40, -50,
                       -30, -20, -10, 0, 0, -10, -20, -30,
                       -30, -10, 20, 30, 30, 20, -10, -30,
                       -30, -10, 30, 40, 40, 30, -10, -30,
                       -30, -10, 30, 40, 40, 30, -10, -30,
                       -30, -10, 20, 30, 30, 20, -10, -30,
                       -30, -30, 0, 0, 0, 0, -30, -30,
                       -50, -30, -30, -30, -30, -30, -30, -50]


### Checking for endgame

The function `is_endgame` takes the current `board` and returns with help of `color_endgame` wether both players match one of the two criteria for the endgame according to the simple evaluation function. This means, the endgame beings if:
1. Both sides have no queens or
2. Every side which has a queen has additionally no other pieces or one minorpiece maximum

(See https://www.chessprogramming.org/Simplified_Evaluation_Function#King).

In [37]:
def is_endgame(board):
    white_endgame = color_endgame(board, chess.WHITE)
    black_endgame = color_endgame(board, chess.BLACK)
    return white_endgame and black_endgame


def color_endgame(board, color):
    return (len(board.pieces(chess.QUEEN, color)) == 0 or
            len(board.pieces(chess.BISHOP, color)) + len(board.pieces(chess.KNIGHT, color)) < 2 and
            len(board.pieces(chess.ROOK, color)) == 0)

### Function for the Piece-Square Tables

This function calculates the value using the `piece_values` (according to the piece-square tables), `player_pieces` and `enemy_pieces` (the list of the player's respectively the opponent's pieces) as well as the current `board` and returns the calculated value.

In [38]:
def piece_square_tables(piece_square_values, player_pieces, enemy_pieces, board):
    val = 0
    black_piece_square_values = piece_square_values
    white_piece_square_values = piece_square_values[::-1]
    for piece in player_pieces:
        if board.turn:
            val += white_piece_square_values[piece]
        else:
            val += black_piece_square_values[piece]
    for piece in enemy_pieces:
        if board.turn:
            val -= black_piece_square_values[piece]
        else:
            val -= white_piece_square_values[piece]
    return val

### Maximum value (32-bit integer)

In [39]:
WIN_VALUE = 2147483647

## Static evaluation

The function `static_eval` takes the parameters `board` (the current board) and `endgame` (if the players are in the endgame) and evaluates the current state of the board. First, the function checks, if the game is over and returns the corresponsing value (e.g. if it is white's turn and white won, it returns the maximum value defined above). If the game is not over, the function computes the total values for each piece type of both sides by multiplying the piece value* with the amount of pieces on the board. Lastly, the function uses the auxiliary function `piece_square_tables` to calculate the values given by the positions of the pieces.

*The piece values are:
* Pawn = 100 cp
* Knight = 320 cp
* Bishop = 330 cp
* Rook = 500 cp
* Queen = 900 cp

`cp` stands for *centipawn* i.e. 0.01 pawn.

In [40]:
def static_eval(board, endgame):
    # STEP ONE: Check, if the game is over
    # White won
    if (board.result() == "1-0" and board.turn or
            board.result() == "0-1" and not board.turn):
        result = WIN_VALUE - board.fullmove_number
        return result
    # Black won
    elif (board.result() == "0-1" and board.turn or
          board.result() == "1-0" and not board.turn):
        result = -WIN_VALUE + board.fullmove_number
        return result
    # Draw
    elif (board.result() == "1/2-1/2" or
          board.is_stalemate() or
          board.is_insufficient_material() or
          board.is_seventyfive_moves() or
          board.is_fivefold_repetition()):
        result = 0
        return result
    # STEP TWO: Calculate piece values using the piece-square tables
    # Value that will be returned eventually
    val = 0
    # Piece values in centipawns as explained above
    piece_values = [100, 320, 330, 500, 900]
    for piece_type, piece_value in enumerate(piece_values):
        piece_type += 1
        player_pieces = board.pieces(piece_type, board.turn)
        enemy_pieces = board.pieces(piece_type, not board.turn)
        val += len(player_pieces) * piece_value
        val -= len(enemy_pieces) * piece_value
        if piece_type == chess.PAWN:
            val += piece_square_tables(PAWN_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.KNIGHT:
            val += piece_square_tables(KNIGHT_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.BISHOP:
            val += piece_square_tables(BISHOP_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.ROOK:
            val += piece_square_tables(ROOK_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.QUEEN:
            val += piece_square_tables(QUEEN_VALUES, player_pieces, enemy_pieces, board)
        if piece_type == chess.KING:
            if endgame:
                val += piece_square_tables(KING_VALUES_ENDGAMAE, player_pieces, enemy_pieces, board)
            else:
                val += piece_square_tables(KING_VALUES_MIDDLEGAME, player_pieces, enemy_pieces, board)
    return val

## Minimax function

The main function, implementing the minimax algorithm (technically Negamax), takes the following parameters:
* `board`, the current board
* `depth`, the depth to which the board will be analyzed
* `alpha` and `beta`, needed for Alpha-Beta-Pruning

The function returns the biggest calculated value i.e. the value for the best possible current move.  
The function also uses a transposition table (here called `cache`) to save computational ressources. It uses the `zobrist_hash` function to create a cache key made out of the hash as well as the current depth. Inside the cache, the following parameters are being saved:
* a `flag` (≤, =, or ≥)
* the current value

First, the function checks, if the key, i.e. the current board and the current depth, are already in the cache. If so, it takes the flag and returns the value accordingly (return the value if the flag equals '=', if it equals '≤' and the value is less than or equal to alpha, or if the flag is '≥' and the value is bigger than or equal to beta, otherwise it calls `alphaBeta`). If the key is not in the cache at all, the function `alphaBeta` will be called directly.

In [41]:
def value(board, limit, alpha=-math.inf, beta=math.inf):
    global MINIMAX_ACCESS
    global CACHE_ACCESS
    MINIMAX_ACCESS += 1
    global Cache
    key = (zobrist_hash(board), limit)
    if key in Cache:

        flag, v = Cache[key]
        if flag == '=':
            CACHE_ACCESS += 1
            return v
        if flag == '≤':
            if v <= alpha:
                CACHE_ACCESS += 1
                return v
            if v < beta:
                w = alphaBeta(board, limit, alpha, v)
                store_cache(board, limit, alpha, v, w)
                return w
            if beta <= v:
                w = alphaBeta(board, limit, alpha, beta)
                store_cache(board, limit, alpha, beta, w)
                return w
        if flag == '≥':
            if beta <= v:
                CACHE_ACCESS += 1
                return v
            if alpha < v:
                w = alphaBeta(board, limit, v, beta)
                store_cache(board, limit, v, beta, w)
                return w
            if v <= alpha:
                w = alphaBeta(board, limit, alpha, beta)
                store_cache(board, limit, alpha, beta, w)
                return w
    else:
        v = alphaBeta(board, limit, alpha, beta)
        store_cache(board, limit, alpha, beta, v)
        return v

The function `store_cache` takes the current `board`, `limit`, `alpha`, `beta`, and the value (`val`) and stores the value accordingly in the cache.

In [42]:
def store_cache(board, limit, alpha, beta, val):
    global Cache
    key = (zobrist_hash(board), limit)
    if val <= alpha:
        Cache[key] = ('≤', val)
    elif val < beta:
        Cache[key] = ('=', val)
    else:
        Cache[key] = ('≥', val)

The function `value_cache` takes the current `board` and the `limit` and returns the value from the cache, if it exists.

In [43]:
def value_cache(board, limit):
    _, val = Cache.get((board, limit), ('=', 0))
    return val

The function `alphaBeta` takes the same parameters as `value` and then calculates and returns the best value using minimax.

In [44]:
def alphaBeta(board, limit, alpha=-math.inf, beta=math.inf):
    if not board.legal_moves or limit == 0:
        return static_eval(board, is_endgame(board))
    val = alpha
    next_moves = board.legal_moves
    moves = []  # empty priority queue
    cnt = 0
    for nm in next_moves:
        cnt += 1
        # no '-' in front of value as smallest value has highest priority
        # limit-3 is the value from the previous iteration
        heapq.heappush(moves, (value_cache(nm, limit - 3), cnt, nm))
    while moves != []:
        _, _, nm = heapq.heappop(moves)
        board.push(nm)
        val = max(val, -value(board, limit - 1, -beta, -alpha))
        board.pop()
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

The function `best_move` takes the current `board` and the search `limit` and returns the best possible move. It is the function to be called by `minimax_input`.

In [45]:
def best_move(board, limit):
    nm = board.legal_moves
    best_val = value(board, limit)
    best_moves = []
    for m in nm:
        board.push(m)
        if -value(board, limit - 1) == best_val:
            best_moves.append(m)
        board.pop()
    if best_moves:
        best_state = random.choice(best_moves)
        return best_val, best_state, False
    return 0, None, True

### Input function for minimax
This function is the first to call; it sets the `cache`, initialises some debug/logging parameters and checks, if the move is covered by the opening library (`get_polyglot_move`). The function takes the `board` and the desired `depth`.

In [46]:
BEST_MOVE = None


def minimax_input(board, limit):
    # DEBUG/LOGGING
    global CACHE_ACCESS
    global MINIMAX_ACCESS
    CACHE_ACCESS = 1
    MINIMAX_ACCESS = 1
    global Cache
    Cache = {}
    val, polyglot_move = get_polyglot_move(board)
    if polyglot_move:
        return val, polyglot_move, False
    return best_move(board, limit)

### Using the opening library
This auxiliary function takes the current `board` and checks if the played move is in an external opening library. It then chooses a random move, proposed by the library and returns the value 0 and the `move`.

In [47]:
POLYGLOT_PATH = "data/polyglot/performance.bin"


def get_polyglot_move(board):
    with chess.polyglot.open_reader(POLYGLOT_PATH) as reader:
        l = list(reader.find_all(board))
        if l:
            return 0, random.choice(l).move
        return 0, None